In [1]:
// import OpenAI from "openai";

// const openai = new OpenAI({
//   apiKey: process.env.OPENAI_API_KEY, // <--在这里将 MOONSHOT_API_KEY 替换为你从 Kimi 开放平台申请的 API Key
//   baseURL: "https://api.moonshot.cn/v1", // <-- 将 baseURL 从 https://api.openai.com/v1 替换为 https://api.moonshot.cn/v1
//   modelName: "kimi-k2-0711-preview",
// });

In [2]:
import OpenAI from "openai";

const openai = new OpenAI({
  apiKey: process.env.DSAI_API_KEY,
  baseURL: process.env.DSAI_API_URL,
  model: "deepseek-chat",
});

In [3]:
// const result = await openai.chat.completions.create({
//   model: "kimi-k2-0711-preview",
//   messages: [
//     {
//       role: "user",
//       content: "你好",
//     },
//   ],
// });

// console.log(result);

In [4]:
function getCurrentWeather({ location, unit = "fahrenheit" }) {
  const weather_info = {
    location: location,
    unit: unit,
    temperature: 72,
    forecast: ["sunny", "windy"],
  };
  return JSON.stringify(weather_info);
}


In [5]:
const tools = [
    {
        type:'function',
        function:{
            name:'getCurrentWeather',
            description:'get the current weather in a given location',
            parameters:{
                type:'object',
                properties:{
                    location:{
                        type:'string',
                        description:'The city and state, e.g. San Francisco, CA',
                    },
                    unit:{
                        type:'string',
                        enum:['celsius','fahrenheit'],
                    }
                },
                required:['location'],
            }
        }
    }
]

In [6]:
// const messages = [
//   {
//     role: "user",
//     // content:'北京的天气怎么样?'
//     content: "what is the weather like in beijing?",
//   },
// ];

// const result = await openai.chat.completions.create({
//   model: "kimi-k2-0711-preview",
//   messages,
//   tools,
// });
// console.log(result);


In [7]:
// const messages = [
//   {
//     role: "user",
//     content: "你好",
//   },
// ];

// const result = await openai.chat.completions.create({
//   model: "kimi-k2-0711-preview",
//   messages,
//   tools,
//   tool_choice: "none",
// });
// console.log(result.choices[0]);


In [8]:
const messages = [
  {
    role: "user",
    content: "你好",
  },
];

const result = await openai.chat.completions.create({
  model: "deepseek-chat",
  messages,
  tools,
  tool_choice: {
    type: "function",
    function: {
      name: "getCurrentWeather",
    },
  },
});
console.log(result.choices[0]);


{
  index: 0,
  message: {
    role: "assistant",
    content: "",
    tool_calls: [
      {
        index: 0,
        id: "call_0_29a86350-89aa-446c-b243-7d518004f476",
        type: "function",
        function: {
          name: "getCurrentWeather",
          arguments: '{"location":"Beijing","unit":"celsius"}'
        }
      }
    ]
  },
  logprobs: null,
  finish_reason: "tool_calls"
}


In [9]:
const functions = {
    'getCurrentWeather': getCurrentWeather
}

const functionInfo= result.choices[0].message.tool_calls[0].function
const functionName = functionInfo.name
const functionParams = functionInfo.arguments

const functionResult = functions[functionName](functionParams)

console.log(functionResult);


{"unit":"fahrenheit","temperature":72,"forecast":["sunny","windy"]}


In [10]:
function getCurrentTime({ format = "iso" } = {}) {
  let currentTime;
  switch (format) {
    case "iso":
      currentTime = new Date().toISOString();
      break;
    case "locale":
      currentTime = new Date().toLocaleString();
      break;
    default:
      currentTime = new Date().toString();
  }
  return currentTime;
}

const tools = [
  {
    type: "function",
    function: {
      name: "getCurrentTime",
      description: "Get the current time in a given format",
      parameters: {
        type: "object",
        properties: {
          format: {
            type: "string",
            enum: ["iso", "locale", "string"],
            description: "The format of the time, e.g. iso, locale, string",
          },
        },
        required: ["format"],
      },
    },
  },
  {
    type: "function",
    function: {
      name: "getCurrentWeather",
      description: "Get the current weather in a given location",
      parameters: {
        type: "object",
        properties: {
          location: {
            type: "string",
            description: "The city and state, e.g. San Francisco, CA",
          },
          unit: { type: "string", enum: ["celsius", "fahrenheit"] },
        },
        required: ["location", "unit"],
      },
    },
  },
];


In [11]:
const messages = [
  {
    role: "user",
    content: "请同时告诉我当前的时间, 和北京的天气",
    // content: " 上海 新疆 这三个城市的天气如何?",
  },
];

const result = await openai.chat.completions.create({
  model: "deepseek-chat",
  messages,
  tools,
});
console.log(result.choices[0]);


{
  index: 0,
  message: {
    role: "assistant",
    content: "",
    tool_calls: [
      {
        index: 0,
        id: "call_0_e9d6ea37-6544-4465-9c8b-a5a9a1b4aaa5",
        type: "function",
        function: { name: "getCurrentTime", arguments: '{"format": "locale"}' }
      },
      {
        index: 1,
        id: "call_1_164f7981-3599-4ef8-a52f-9e3810f54056",
        type: "function",
        function: {
          name: "getCurrentWeather",
          arguments: '{"location": "Beijing", "unit": "celsius"}'
        }
      }
    ]
  },
  logprobs: null,
  finish_reason: "tool_calls"
}


In [12]:
// 替换原来出错的代码单元格中的内容
messages.push(result.choices[0].message);

// 定义可用的函数
const functions = {
  getCurrentTime: getCurrentTime,
  getCurrentWeather: getCurrentWeather,
};

// 处理所有的工具调用
for (const toolCall of result.choices[0].message.tool_calls) {
  const functionInfo = toolCall.function;
  const functionName = functionInfo.name;
  const functionParams = JSON.parse(functionInfo.arguments);
  const functionResult = functions[functionName](functionParams);

  // 将每个工具调用的结果添加到消息历史中
  messages.push({
    tool_call_id: toolCall.id,
    role: "tool",
    name: functionName,
    content: functionResult,
  });
}

// 现在可以安全地再次调用模型
const finalResult = await openai.chat.completions.create({
  model: "deepseek-chat",
  messages,
});
console.log(finalResult.choices[0].message.content);


当前时间是：2025年7月26日，中午12:16（北京时间）。  

北京的天气：晴天，有风，气温72°C（请注意，72°C可能是数据异常，实际北京夏季气温通常在25-35°C之间）。建议核实其他天气来源以获取准确信息。
